# ```stockPrice_year```
- - 아래의 파일을 병합하여 분석
  - ```KCGS_ESGRating.csv```
  - ```KRX_KOSPI200_stock_year.csv```

# import

In [ ]:
import os
import sys
import time
import pickle
import warnings
import urllib.request
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import requests
from bs4 import BeautifulSoup as bs

import FinanceDataReader as fdr
from tqdm import tqdm

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
# pd.options.display.float_format = '{:.4f}'.format
plt.style.use("ggplot")
%config InlineBackend.figure_format = 'retina'

sys.path.append("../import")
import module as m

data_path = m.data_path
fp_stock = f"{data_path}{m.fp["stockPrice"]}"
fp_esg = f"{data_path}{m.fp["stockPrice"]}"
fp_fs = f"{data_path}{m.fp["stockPrice"]}"
fp_ip = f"{data_path}indexPrice.parquet"
fp_stock_year = f"{data_path}{stockPrice_year.csv"

# ```stockPrice```

## Data Load

In [115]:
df_stock = pd.read_csv(fp_stock, index_col=False)
module.reduce_mem_usage(df_stock)
# 종목코드 자리수
df_stock["종목코드"] = df_stock["종목코드"].apply(m.six_digit)
df_stock

Mem. usage decreased to 97.90 Mb (38.0% reduction)


,종목코드,종목명,연_월_일,시가,고가,저가,종가,거래량,등락률,연,분기,월,일,연_분기,연_월,분기_월,월_일,연_분기_월,최대가격,최소가격,시가_mmscl,고가_mmscl,저가_mmscl,종가_mmscl
0,000020,동화약품,2011-01-03,5320,5370,5220,5280,78725,-0.0019,2011,1,1,3,2011-1,2011-1,1-1,1-3,2011-1-1,34450,3860,0.0477,0.0494,0.0445,0.0464
1,000020,동화약품,2011-01-04,5250,5330,5230,5300,58101,0.0038,2011,1,1,4,2011-1,2011-1,1-1,1-4,2011-1-1,34450,3860,0.0454,0.0481,0.0448,0.0471
2,000020,동화약품,2011-01-05,5300,5350,5220,5240,78870,-0.0113,2011,1,1,5,2011-1,2011-1,1-1,1-5,2011-1-1,34450,3860,0.0471,0.0487,0.0445,0.0451
3,000020,동화약품,2011-01-06,5240,5250,5070,5110,174303,-0.0248,2011,1,1,6,2011-1,2011-1,1-1,1-6,2011-1-1,34450,3860,0.0451,0.0454,0.0396,0.0409
4,000020,동화약품,2011-01-07,5110,5150,5080,5090,36832,-0.0039,2011,1,1,7,2011-1,2011-1,1-1,1-7,2011-1-1,34450,3860,0.0409,0.0422,0.0399,0.0402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862629,402340,SK스퀘어,2023-01-26,36600,37450,36550,37400,651019,0.0177,2023,1,1,26,2023-1,2023-1,1-1,1-26,2023-1-1,85000,31100,0.1021,0.1178,0.1011,0.1169
862630,402340,SK스퀘어,2023-01-27,37150,38050,37000,37700,550261,0.0080,2023,1,1,27,2023-1,2023-1,1-1,1-27,2023-1-1,85000,31100,0.1122,0.1289,0.1094,0.1224
862631,402340,SK스퀘어,2023-01-30,37850,38150,37250,37550,367640,-0.0040,2023,1,1,30,2023-1,2023-1,1-1,1-30,2023-1-1,85000,31100,0.1252,0.1307,0.1141,0.1197
862632,402340,SK스퀘어,2023-01-31,37550,37850,36250,36250,959658,-0.0346,2023,1,1,31,2023-1,2023-1,1-1,1-31,2023-1-1,85000,31100,0.1197,0.1252,0.0955,0.0955


## 연평균

In [14]:
dict_agg = {
    "시가": "mean",
    "고가": "max",
    "저가": "min",
    "종가": "mean",
    "거래량": "sum",
    "시가_mmscl": "mean",
    "고가_mmscl": "max",
    "저가_mmscl": "min",
    "종가_mmscl": "mean",
}

df_stock_year = df_stock.groupby(["종목코드", "종목명", "연"]).agg(dict_agg)
df_stock_year = df_stock_year.reset_index()
df_stock_year.head()

,종목코드,종목명,연,시가,고가,저가,종가,거래량,시가_mmscl,고가_mmscl,저가_mmscl,종가_mmscl
0,000020,동화약품,2011,4776.6734,6470,3860,4764.1532,35915126,0.0300,0.0853,0.0000,0.0296
1,000020,동화약품,2012,5344.4153,6800,4060,5341.6734,32035524,0.0485,0.0961,0.0065,0.0484
2,000020,동화약품,2013,6121.6599,7420,4315,6117.0243,20256240,0.0739,0.1164,0.0149,0.0738
3,000020,동화약품,2014,5496.8571,6560,4410,5508.2653,20925838,0.0535,0.0883,0.0180,0.0539
4,000020,동화약품,2015,7659.5968,12250,5400,7661.3710,54304424,0.1242,0.2742,0.0504,0.1243


# merge_esgRating

## Data Load

In [27]:
df_esg = pd.read_csv(fp_esg, index_col=False)
df_esg["종목코드"] = df_esg["종목코드"].apply(six_digit)
display(df_esg.head())

,종목코드,종목명,평가년도,ESG종합,E,S,G,ESG종합_enc,E_enc,S_enc,G_enc
0,000010,신한은행,2022,NaN,NaN,NaN,B+,NaN,NaN,NaN,3.0000
1,000020,동화약품,2011,B,B+,B,B,2.0000,3.0000,2.0000,2.0000
2,000020,동화약품,2012,B,B+,B,B,2.0000,3.0000,2.0000,2.0000
3,000020,동화약품,2013,B,B,B,B,2.0000,2.0000,2.0000,2.0000
4,000020,동화약품,2014,B,B,B,B,2.0000,2.0000,2.0000,2.0000


## 전처리

In [28]:
def tmp(x):
    try:
        dict_enc = {
            "S": "6",
            "A+": "5",
            "A": "4",
            "B+": "3",
            "B+ 이하": "3",
            "B": "2",
            "B이하": "2",
            "C": "1",
            "D": "0",
            "-": np.nan,
            "": np.nan,
            "NaN": np.nan,
        }
        return dict_enc[x]
    except:
        return np.nan


df_esg["ESG종합_enc"] = df_esg["ESG종합"].fillna(np.nan).apply(tmp)
df_esg["E_enc"] = df_esg["E"].fillna(np.nan).apply(tmp)
df_esg["S_enc"] = df_esg["S"].fillna(np.nan).apply(tmp)
df_esg["G_enc"] = df_esg["G"].fillna(np.nan).apply(tmp)

# df_esg["연"] = df_esg["평가년도"]
df_esg.rename(columns={"평가년도": "연"}, inplace=True)

display(df_esg.head())

,종목코드,종목명,연,ESG종합,E,S,G,ESG종합_enc,E_enc,S_enc,G_enc
0,000010,신한은행,2022,NaN,NaN,NaN,B+,NaN,NaN,NaN,3
1,000020,동화약품,2011,B,B+,B,B,2,3,2,2
2,000020,동화약품,2012,B,B+,B,B,2,3,2,2
3,000020,동화약품,2013,B,B,B,B,2,2,2,2
4,000020,동화약품,2014,B,B,B,B,2,2,2,2


### 병합

In [29]:
df_Stock_year_esg = pd.merge(df_esg, df_stock_year, how='left', on=["종목코드",'연'])
df_Stock_year_esg

,종목코드,종목명_x,연,ESG종합,E,S,G,ESG종합_enc,E_enc,S_enc,G_enc,종목명_y,시가,고가,저가,종가,거래량,시가_mmscl,고가_mmscl,저가_mmscl,종가_mmscl
0,000010,신한은행,2022,NaN,NaN,NaN,B+,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000020,동화약품,2011,B,B+,B,B,2,3,2,2,동화약품,4776.6734,6470.0000,3860.0000,4764.1532,35915126.0000,0.0300,0.0853,0.0000,0.0296
2,000020,동화약품,2012,B,B+,B,B,2,3,2,2,동화약품,5344.4153,6800.0000,4060.0000,5341.6734,32035524.0000,0.0485,0.0961,0.0065,0.0484
3,000020,동화약품,2013,B,B,B,B,2,2,2,2,동화약품,6121.6599,7420.0000,4315.0000,6117.0243,20256240.0000,0.0739,0.1164,0.0149,0.0738
4,000020,동화약품,2014,B,B,B,B,2,2,2,2,동화약품,5496.8571,6560.0000,4410.0000,5508.2653,20925838.0000,0.0535,0.0883,0.0180,0.0539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6654,None,대덕GDS,2018,B+,B,B+,B+,3,2,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6655,None,골든브릿지증권,2018,B,B+,B,B,2,3,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6656,None,고려개발,2018,B,B,B+,B,2,2,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6657,None,계룡건설산업,2018,B,B,B+,B,2,2,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 전처리

In [30]:
df_Stock_year_esg.drop(["종목명_y"], axis="columns", inplace=True)
d = {
    "종목명_x": "종목명",
    "시가": "시가_연평균",
    "고가": "고가_연",
    "저가": "저가_연",
    "종가": "종가_연평균",
    "시가_mmscl": "시가_mmscl_연평균",
    "고가_mmscl": "고가_mmscl_연",
    "저가_mmscl": "저가_mmscl_연",
    "종가_mmscl": "종가_mmscl_연평균",
}
df_Stock_year_esg.rename(columns=d, inplace=True)
df_Stock_year_esg

,종목코드,종목명,연,ESG종합,E,S,G,ESG종합_enc,E_enc,S_enc,G_enc,시가_연평균,고가_연,저가_연,종가_연평균,거래량,시가_mmscl_연평균,고가_mmscl_연,저가_mmscl_연,종가_mmscl_연평균
0,000010,신한은행,2022,NaN,NaN,NaN,B+,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000020,동화약품,2011,B,B+,B,B,2,3,2,2,4776.6734,6470.0000,3860.0000,4764.1532,35915126.0000,0.0300,0.0853,0.0000,0.0296
2,000020,동화약품,2012,B,B+,B,B,2,3,2,2,5344.4153,6800.0000,4060.0000,5341.6734,32035524.0000,0.0485,0.0961,0.0065,0.0484
3,000020,동화약품,2013,B,B,B,B,2,2,2,2,6121.6599,7420.0000,4315.0000,6117.0243,20256240.0000,0.0739,0.1164,0.0149,0.0738
4,000020,동화약품,2014,B,B,B,B,2,2,2,2,5496.8571,6560.0000,4410.0000,5508.2653,20925838.0000,0.0535,0.0883,0.0180,0.0539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6654,None,대덕GDS,2018,B+,B,B+,B+,3,2,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6655,None,골든브릿지증권,2018,B,B+,B,B,2,3,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6656,None,고려개발,2018,B,B,B+,B,2,2,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6657,None,계룡건설산업,2018,B,B,B+,B,2,2,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_Stock_year_esg.dropna(axis=0, subset=['종가_연평균'], inplace=True)
df_Stock_year_esg["거래량"] = df_Stock_year_esg["거래량"].astype(int)
df_Stock_year_esg["고가_연"] = df_Stock_year_esg["고가_연"].astype(int)
df_Stock_year_esg["저가_연"] = df_Stock_year_esg["저가_연"].astype(int)
df_Stock_year_esg

,종목코드,종목명,연,ESG종합,E,S,G,ESG종합_enc,E_enc,S_enc,G_enc,시가_연평균,고가_연,저가_연,종가_연평균,거래량,시가_mmscl_연평균,고가_mmscl_연,저가_mmscl_연,종가_mmscl_연평균
1,000020,동화약품,2011,B,B+,B,B,2,3,2,2,4776.6734,6470,3860,4764.1532,35915126,0.0300,0.0853,0.0000,0.0296
2,000020,동화약품,2012,B,B+,B,B,2,3,2,2,5344.4153,6800,4060,5341.6734,32035524,0.0485,0.0961,0.0065,0.0484
3,000020,동화약품,2013,B,B,B,B,2,2,2,2,6121.6599,7420,4315,6117.0243,20256240,0.0739,0.1164,0.0149,0.0738
4,000020,동화약품,2014,B,B,B,B,2,2,2,2,5496.8571,6560,4410,5508.2653,20925838,0.0535,0.0883,0.0180,0.0539
5,000020,동화약품,2015,B,C,B,B,2,1,2,2,7659.5968,12250,5400,7661.3710,54304424,0.1242,0.2742,0.0504,0.1243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5186,326030,SK바이오팜,2022,A,B+,A,A,4,3,4,4,77132.9268,98700,50900,77126.4228,31475881,0.1200,0.2186,0.0000,0.1200
5195,329180,현대중공업,2022,B+,B+,A,B+,3,3,4,3,120650.0000,161500,92700,120801.2195,65853984,0.4312,1.0000,0.0418,0.4331
5196,336260,두산퓨얼셀,2022,B+,B+,A+,B,3,3,5,2,35066.0569,48400,23650,34959.1463,126613009,0.5063,0.7236,0.3208,0.5049
5200,352820,하이브,2022,C,D,D,B,1,0,0,2,202105.6911,360500,107000,201534.5528,68959394,0.3025,0.8062,0.0000,0.3005


## merge_finaStat

### Data Load

In [35]:
df_fs = pd.read_csv(fp_fs, index_col=False)
# 종목코드 자리수
df_fs["종목코드"] = df_fs["종목코드"].apply(six_digit)
df_fs.head(2)

,종목코드,종목명,연_월_일,재무제표구분코드명,기업매출금액,기업영업이익,포괄손익계산금액,기업당기순이익,기업총자산금액,기업총부채금액,기업총자본금액,기업자본금액,재무제표부채비율,연,분기,월,연_분기,연_월,분기_월,연_분기_월,기업매출금액_mmscl,기업영업이익_mmscl,포괄손익계산금액_mmscl,기업당기순이익_mmscl,기업총자산금액_mmscl,기업총부채금액_mmscl,기업총자본금액_mmscl,기업자본금액_mmscl,재무제표부채비율_mmscl
0,000020,동화약품,20151231,별도요약재무제표,223201285434,4812973681,6000622879,5608652157,317187030052,87069287627,230117742425,27931470000,37.8368,2015,4,12,2015-4,2015-12,4-12,2015-4-12,0.0000,0.0000,0.0000,0.0000,0.0000,0.8054,0.0000,NaN,1.0000
1,000020,동화약품,20161231,별도요약재무제표,237470834801,11259333902,35655076190,26254318411,324604536650,71679236748,252925299902,27931470000,28.3401,2016,4,12,2016-4,2016-12,4-12,2016-4-12,0.1700,0.3515,0.5008,0.4987,0.0568,0.0671,0.1754,NaN,0.3848


### 연평균

In [36]:
dict_agg = {}
l = [
    "기업매출금액",
    "기업영업이익",
    "포괄손익계산금액",
    "기업당기순이익",
    "기업총자산금액",
    "기업총부채금액",
    "기업총자본금액",
    "기업자본금액",
    "재무제표부채비율",
    "기업매출금액_mmscl",
    "기업영업이익_mmscl",
    "포괄손익계산금액_mmscl",
    "기업당기순이익_mmscl",
    "기업총자산금액_mmscl",
    "기업총부채금액_mmscl",
    "기업총자본금액_mmscl",
    "기업자본금액_mmscl",
    "재무제표부채비율_mmscl",
]


for i in l:
    dict_agg[i] = "mean"

df_fs_year = df_fs.groupby(["종목코드", "종목명", "연", "재무제표구분코드명"]).agg(dict_agg)
df_fs_year = df_fs_year.reset_index()
df_fs_year.head()

,종목코드,종목명,연,재무제표구분코드명,기업매출금액,기업영업이익,포괄손익계산금액,기업당기순이익,기업총자산금액,기업총부채금액,기업총자본금액,기업자본금액,재무제표부채비율,기업매출금액_mmscl,기업영업이익_mmscl,포괄손익계산금액_mmscl,기업당기순이익_mmscl,기업총자산금액_mmscl,기업총부채금액_mmscl,기업총자본금액_mmscl,기업자본금액_mmscl,재무제표부채비율_mmscl
0,000020,동화약품,2015,별도요약재무제표,223201285434.0000,4812973681.0000,6000622879.0000,5608652157.0000,317187030052.0000,87069287627.0000,230117742425.0000,27931470000.0000,37.8368,0.0000,0.0000,0.0000,0.0000,0.0000,0.8054,0.0000,NaN,1.0000
1,000020,동화약품,2016,별도요약재무제표,237470834801.0000,11259333902.0000,35655076190.0000,26254318411.0000,324604536650.0000,71679236748.0000,252925299902.0000,27931470000.0000,28.3401,0.1700,0.3515,0.5008,0.4987,0.0568,0.0671,0.1754,NaN,0.3848
2,000020,동화약품,2017,별도요약재무제표,258881616575.0000,10987308187.0000,65218742497.0000,47009013175.0000,367225133428.0000,70280404999.0000,296944728429.0000,27931470000.0000,23.6678,0.4250,0.3367,1.0000,1.0000,0.3831,0.0000,0.5140,NaN,0.0822
3,000020,동화약품,2018,별도요약재무제표,306602589029.0000,11232142004.0000,14545424343.0000,10074474538.0000,370294498762.0000,73197485231.0000,297097013531.0000,27931470000.0000,24.6376,0.9935,0.3500,0.1443,0.1079,0.4066,0.1399,0.5152,NaN,0.1450
4,000020,동화약품,2018,연결요약재무제표,306602589029.0000,11225780035.0000,14539062374.0000,10068112569.0000,370599242793.0000,73198591231.0000,297400651562.0000,27931470000.0000,24.6128,0.9935,0.3497,0.1442,0.1077,0.4089,0.1400,0.5175,NaN,0.1434


### 병합

In [37]:
df_Stock_year_esgFn = pd.merge(df_Stock_year_esg, df_fs_year, how='left', on=["종목코드",'연'])
df_Stock_year_esgFn.drop(['종목명_y'], axis='columns', inplace=True)
df_Stock_year_esgFn.rename(columns={'종목명_x':'종목명'}, inplace=True)
df_Stock_year_esgFn

,종목코드,종목명,연,ESG종합,E,S,G,ESG종합_enc,E_enc,S_enc,G_enc,시가_연평균,고가_연,저가_연,종가_연평균,거래량,시가_mmscl_연평균,고가_mmscl_연,저가_mmscl_연,종가_mmscl_연평균,재무제표구분코드명,기업매출금액,기업영업이익,포괄손익계산금액,기업당기순이익,기업총자산금액,기업총부채금액,기업총자본금액,기업자본금액,재무제표부채비율,기업매출금액_mmscl,기업영업이익_mmscl,포괄손익계산금액_mmscl,기업당기순이익_mmscl,기업총자산금액_mmscl,기업총부채금액_mmscl,기업총자본금액_mmscl,기업자본금액_mmscl,재무제표부채비율_mmscl
0,000020,동화약품,2011,B,B+,B,B,2,3,2,2,4776.6734,6470,3860,4764.1532,35915126,0.0300,0.0853,0.0000,0.0296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000020,동화약품,2012,B,B+,B,B,2,3,2,2,5344.4153,6800,4060,5341.6734,32035524,0.0485,0.0961,0.0065,0.0484,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000020,동화약품,2013,B,B,B,B,2,2,2,2,6121.6599,7420,4315,6117.0243,20256240,0.0739,0.1164,0.0149,0.0738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000020,동화약품,2014,B,B,B,B,2,2,2,2,5496.8571,6560,4410,5508.2653,20925838,0.0535,0.0883,0.0180,0.0539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000020,동화약품,2015,B,C,B,B,2,1,2,2,7659.5968,12250,5400,7661.3710,54304424,0.1242,0.2742,0.0504,0.1243,별도요약재무제표,223201285434.0000,4812973681.0000,6000622879.0000,5608652157.0000,317187030052.0000,87069287627.0000,230117742425.0000,27931470000.0000,37.8368,0.0000,0.0000,0.0000,0.0000,0.0000,0.8054,0.0000,NaN,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2733,326030,SK바이오팜,2022,A,B+,A,A,4,3,4,4,77132.9268,98700,50900,77126.4228,31475881,0.1200,0.2186,0.0000,0.1200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2734,329180,현대중공업,2022,B+,B+,A,B+,3,3,4,3,120650.0000,161500,92700,120801.2195,65853984,0.4312,1.0000,0.0418,0.4331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2735,336260,두산퓨얼셀,2022,B+,B+,A+,B,3,3,5,2,35066.0569,48400,23650,34959.1463,126613009,0.5063,0.7236,0.3208,0.5049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2736,352820,하이브,2022,C,D,D,B,1,0,0,2,202105.6911,360500,107000,201534.5528,68959394,0.3025,0.8062,0.0000,0.3005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 영속화

In [38]:
df_Stock_year_esgFn.to_csv(fp_stock_year, index=False)
glob(fp_stock_year)

['../data/stockPrice_year.csv']

In [39]:
fp = fp_stock_year[:-4]
fp += ".pickle"

with open(f"{fp}.pickle","wb") as f:
    pickle.dump(df_Stock_year_esgFn, f)
glob(f"{fp}.pickle")

['../data/stockPrice_year.csv.pickle']

In [109]:
def DfPrst(df: pd.DataFrame, file: str = "../data/temp", *format: str):
    """
    df : pandas.DataFrame
    prst : persistence
    pd.df를 영속화하는 함수
    """

    if "csv" in format:
        f = file + ".csv"
        df.to_csv(f, index=False)
        print(glob(f))

    if "pickle" in format:
        f = file + ".pickle"
        df.to_pickle(f)
        print(glob(f))

    if "parquet" in format:
        f = file + ".parquet"
        df.to_parquet(f, index=False)
        print(glob(f))


df_esg = pd.read_csv(fp_esg, index_col=False)
DfPrst(df_esg, "../data/temp", "csv", "pickle", "parquet")

['../data/temp.csv']
['../data/temp.pickle']
['../data/temp.parquet']


In [110]:
def DataLoad(file: str):

    fm = file.split(".")[-1]

    if "csv" == fm:

        df = pd.read_csv(file, index=False)
        df = reduce_mem_usage(df)

        # df["종목코드"] = df["종목코드"].astype(str).apply(lambda x: x.zfill(6))
        # df["stock_code"] = df["stock_code"].astype(str).apply(lambda x: x.zfill(6))

        return df

    if "pickle" == fm:
        return reduce_mem_usage(pd.read_pickle(file))

    if "parquet" == fm:
        return reduce_mem_usage(pd.read_parquet(file))

In [111]:
a = DataLoad("../data/temp.parquet")
a

Mem. usage decreased to  0.34 Mb (38.6% reduction)


,종목코드,종목명,평가년도,ESG종합,E,S,G,ESG종합_enc,E_enc,S_enc,G_enc
0,10.0000,신한은행,2022,None,None,None,B+,NaN,NaN,NaN,3.0000
1,20.0000,동화약품,2011,B,B+,B,B,2.0000,3.0000,2.0000,2.0000
2,20.0000,동화약품,2012,B,B+,B,B,2.0000,3.0000,2.0000,2.0000
3,20.0000,동화약품,2013,B,B,B,B,2.0000,2.0000,2.0000,2.0000
4,20.0000,동화약품,2014,B,B,B,B,2.0000,2.0000,2.0000,2.0000
...,...,...,...,...,...,...,...,...,...,...,...
6654,NaN,대덕GDS,2018,B+,B,B+,B+,3.0000,2.0000,3.0000,3.0000
6655,NaN,골든브릿지증권,2018,B,B+,B,B,2.0000,3.0000,2.0000,2.0000
6656,NaN,고려개발,2018,B,B,B+,B,2.0000,2.0000,3.0000,2.0000
6657,NaN,계룡건설산업,2018,B,B,B+,B,2.0000,2.0000,3.0000,2.0000


In [104]:
a

# ESG와 연평균주식가격 분석

## Data Load

In [40]:
if glob(f"{fp_stock_year}.pickle"):
    with open(f"{fp_stock_year}.pickle","rb") as f:
        df_Stock_year_esgFn = pickle.load(f)
        
elif glob(fp_stock_year):
    df_Stock_year_esgFn = pd.read_csv(fp_stock_year, index_col=False)
    df_Stock_year_esgFn["종목코드"] = df_Stock_year_esgFn["종목코드"].apply(six_digit)
    
    def tmp(x):
        try:
            dict_enc = {
                "S": "6",
                "A+": "5",
                "A": "4",
                "B+": "3",
                "B+ 이하": "3",
                "B": "2",
                "B이하": "2",
                "C": "1",
                "D": "0",
                "-": np.nan,
                "": np.nan,
                "NaN": np.nan,
            }
            return dict_enc[x]
        except:
            return np.nan


    df_Stock_year_esgFn["ESG종합_enc"] = df_Stock_year_esgFn["ESG종합"].fillna(np.nan).apply(tmp)
    df_Stock_year_esgFn["E_enc"] = df_Stock_year_esgFn["E"].fillna(np.nan).apply(tmp)
    df_Stock_year_esgFn["S_enc"] = df_Stock_year_esgFn["S"].fillna(np.nan).apply(tmp)
    df_Stock_year_esgFn["G_enc"] = df_Stock_year_esgFn["G"].fillna(np.nan).apply(tmp)

display(df_Stock_year_esgFn.head())

AttributeError: 'str' object has no attribute 'head'

In [41]:
df_Stock_year_esgFn

'../data/stockPrice_year.csv'

In [ ]:

glob(f"{fp_stock_year}.pickle")